In [2]:
### lrasmy@Zhilab last revised November 19 2022   ##
###############################################

In [3]:
### Mount your google drive ###
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import numpy as np
import pandas as pd
import sys
sys.path.insert(0,"/content/drive/MyDrive/pytorch_ehr/Pytorch_EHR_Tutorial/Data_Prep/")
import mimic4_preprocess_util
from mimic4_preprocess_util import *

mimic4_path='/content/drive/MyDrive/pytorch_ehr/Pytorch_EHR_Tutorial/Data_Prep/data/physionet.org/files/mimiciv/2.2'
verbose= True
output_path='/content/drive/MyDrive/pytorch_ehr/Pytorch_EHR_Tutorial/Data_Prep/data/extracted_data'
if not os.path.exists(output_path):
        os.makedirs(output_path)

In [5]:
from importlib import reload
reload(mimic4_preprocess_util)
from mimic4_preprocess_util import *


## Import data needed
- required data already downloaded

In [6]:
stays = read_icustays_table(mimic4_path)
pats = read_patients_table(mimic4_path)
admits = read_admissions_table(mimic4_path)
diagnoses_icd = pd.read_csv(mimic4_path+'/hosp/diagnoses_icd.csv.gz', header=0)
inputevents = pd.read_csv(mimic4_path+'/icu/inputevents.csv.gz', header=0)
omr = pd.read_csv(mimic4_path+'/hosp/omr.csv.gz', header=0)


In [7]:
stays = read_icustays_table(mimic4_path)
stays=stays.reset_index()
if verbose:
    print('START:', stays.stay_id.unique().shape[0], stays.hadm_id.unique().shape[0],
          stays.subject_id.unique().shape[0])
print(min(stays['subject_id']),max(stays['subject_id']))
stays = clean_stays(stays)
print(min(stays['subject_id']),max(stays['subject_id']))
admits = read_admissions_table(mimic4_path)
stays = merge_on_subject_admission(stays, admits)
print(min(stays['subject_id']),max(stays['subject_id']))

patients = read_patients_table(mimic4_path)

stays = merge_on_subject(stays, patients)

print('latest stay cnts:', stays.stay_id.unique().shape[0], stays.hadm_id.unique().shape[0],
         stays.subject_id.unique().shape[0])
stays = add_age_to_icustays(stays)
stays = add_inunit_mortality_to_icustays(stays)
stays = add_inhospital_mortality_to_icustays(stays)

START: 73181 66239 50920
10000032 19999987
10000032 19999987
10000032 19999987
latest stay cnts: 73181 66239 50920


Data were extracted from the MIMIC IV 2.2 database on patients diagnosed with secondary bone tumors according to the International Classification of Diseases codes：  
* ICD 9 - 198.5
* ICD 10 - C7B.03, C79.5

In [8]:
# Filter diagnoses_icd  for specified ICD-10 codes
filtered_diagnoses_icd = diagnoses_icd[
    ((diagnoses_icd['icd_version'] == 9) & (diagnoses_icd['icd_code'] == '1985')) |
    ((diagnoses_icd['icd_version'] == 10) & (diagnoses_icd['icd_code'].str.startswith('C795'))) |
    ((diagnoses_icd['icd_version'] == 10) & (diagnoses_icd['icd_code'].isin(['C7B.03'])))
]

# Display the filtered DataFrame
print(filtered_diagnoses_icd)

         subject_id   hadm_id  seq_num icd_code  icd_version
1519       10002430  26295318        4    C7951           10
3294       10005012  28371912        1     1985            9
5526       10011126  24701479        2     1985            9
6869       10013653  21018096        5     1985            9
7113       10014234  21494930        4     1985            9
...             ...       ...      ...      ...          ...
4735643    19955909  26783401        6    C7951           10
4739841    19963140  22031564        2     1985            9
4743960    19972371  29626339        2     1985            9
4747893    19982183  26205995        1    C7951           10
4748757    19984860  20466914        4     1985            9

[5876 rows x 5 columns]


In [9]:
num_unique_subject_ids = filtered_diagnoses_icd['subject_id'].nunique()
num_unique_subject_ids

2913

In [10]:
patients_with_bone_tumors = filtered_diagnoses_icd['subject_id'].unique()
len(patients_with_bone_tumors)

2913

Merge with icu stays

In [11]:
filtered_stays_icd0 = merge_on_subject(filtered_diagnoses_icd,stays)

filtered_stays_icd0


,subject_id,hadm_id_x,seq_num,icd_code,icd_version,hadm_id_y,stay_id,first_careunit,last_careunit,intime,...,race,gender,dod,anchor_age,anchor_year,yob,age,mortality_inunit,mortality,mortality_inhospital
0,10002430,26295318,4,C7951,10,26295318,38392119,Coronary Care Unit (CCU),Coronary Care Unit (CCU),2129-06-13 00:43:08,...,WHITE,M,2130-01-11,86,2125,2039,90,0,0,0
1,10031757,28477280,6,1985,9,28477280,30458995,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2137-10-12 22:44:57,...,WHITE,F,2137-10-31,67,2137,2070,67,0,0,0
2,10031757,28477280,6,1985,9,28477280,33244906,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2137-10-15 17:29:21,...,WHITE,F,2137-10-31,67,2137,2070,67,0,0,0
3,10032725,25177949,1,1985,9,20611640,30101877,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2143-03-22 06:42:00,...,BLACK/AFRICAN AMERICAN,F,2143-03-30,38,2143,2105,38,0,0,0
4,10054464,21915212,4,1985,9,20069204,35953056,Coronary Care Unit (CCU),Coronary Care Unit (CCU),2122-05-04 09:48:00,...,WHITE,M,2122-06-11,70,2122,2052,70,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4392,19930554,22219697,2,1985,9,27630195,30516447,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical Intensive Care Unit (MICU),2194-07-26 12:39:16,...,BLACK/AFRICAN AMERICAN,F,2198-08-22,35,2194,2159,35,0,0,0
4393,19930554,26906044,3,1985,9,27630195,30516447,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical Intensive Care Unit (MICU),2194-07-26 12:39:16,...,BLACK/AFRICAN AMERICAN,F,2198-08-22,35,2194,2159,35,0,0,0
4394,19955909,26783401,6,C7951,10,29894050,34000757,Trauma SICU (TSICU),Trauma SICU (TSICU),2180-07-29 18:47:34,...,ASIAN - CHINESE,M,2181-05-16,71,2180,2109,71,0,0,0
4395,19963140,22031564,2,1985,9,22031564,36548852,Trauma SICU (TSICU),Trauma SICU (TSICU),2137-09-13 22:53:55,...,WHITE,M,NaT,68,2137,2069,68,0,0,0


In [12]:
stay_filtered = pd.merge(stays,filtered_diagnoses_icd['subject_id'],  on='subject_id', how='inner')
stay_filtered

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,admittime,dischtime,...,race,gender,dod,anchor_age,anchor_year,yob,age,mortality_inunit,mortality,mortality_inhospital
0,10002430,26295318,38392119,Coronary Care Unit (CCU),Coronary Care Unit (CCU),2129-06-13 00:43:08,2129-06-15 22:51:40,2.922593,2129-06-13 00:00:00,2129-06-24 16:01:00,...,WHITE,M,2130-01-11,86,2125,2039,90,0,0,0
1,10031757,28477280,30458995,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2137-10-12 22:44:57,2137-10-14 17:08:34,1.766400,2137-10-12 22:43:00,2137-10-24 17:30:00,...,WHITE,F,2137-10-31,67,2137,2070,67,0,0,0
2,10031757,28477280,33244906,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2137-10-15 17:29:21,2137-10-17 22:16:51,2.199653,2137-10-12 22:43:00,2137-10-24 17:30:00,...,WHITE,F,2137-10-31,67,2137,2070,67,0,0,0
3,10032725,20611640,30101877,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2143-03-22 06:42:00,2143-03-25 15:05:33,3.349687,2143-03-22 04:59:00,2143-03-25 13:00:00,...,BLACK/AFRICAN AMERICAN,F,2143-03-30,38,2143,2105,38,0,0,0
4,10054464,20069204,35953056,Coronary Care Unit (CCU),Coronary Care Unit (CCU),2122-05-04 09:48:00,2122-05-05 11:44:06,1.080625,2122-05-04 09:01:00,2122-05-12 11:44:00,...,WHITE,M,2122-06-11,70,2122,2052,70,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4392,19930554,27630195,30516447,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical Intensive Care Unit (MICU),2194-07-26 12:39:16,2194-07-28 20:47:23,2.338970,2194-07-26 12:38:00,2194-07-29 17:30:00,...,BLACK/AFRICAN AMERICAN,F,2198-08-22,35,2194,2159,35,0,0,0
4393,19930554,27630195,30516447,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical Intensive Care Unit (MICU),2194-07-26 12:39:16,2194-07-28 20:47:23,2.338970,2194-07-26 12:38:00,2194-07-29 17:30:00,...,BLACK/AFRICAN AMERICAN,F,2198-08-22,35,2194,2159,35,0,0,0
4394,19955909,29894050,34000757,Trauma SICU (TSICU),Trauma SICU (TSICU),2180-07-29 18:47:34,2180-07-30 16:06:00,0.887801,2180-07-28 07:15:00,2180-07-31 14:25:00,...,ASIAN - CHINESE,M,2181-05-16,71,2180,2109,71,0,0,0
4395,19963140,22031564,36548852,Trauma SICU (TSICU),Trauma SICU (TSICU),2137-09-13 22:53:55,2137-09-15 13:24:54,1.604850,2137-09-11 17:47:00,2137-09-21 14:39:00,...,WHITE,M,NaT,68,2137,2069,68,0,0,0


In [13]:
filtered_stays_icd0.nunique()

subject_id              1274
hadm_id_x               2831
seq_num                   21
icd_code                   3
icd_version                2
hadm_id_y               1707
stay_id                 1898
first_careunit             9
last_careunit              9
intime                  1898
outtime                 1898
los                     1895
admittime               1707
dischtime               1707
deathtime                345
race                      29
gender                     2
dod                     1000
anchor_age                65
anchor_year               93
yob                      127
age                       70
mortality_inunit           2
mortality                  2
mortality_inhospital       2
dtype: int64

Based on the article, if one subject admitted for more than one times, only keep the first records for the table.

In [14]:
# Group by 'subject_id' and select the first row for each group
first_intime_per_subject = filtered_stays_icd0.groupby('subject_id')['intime'].first()

# Convert the result to a DataFrame
first_intime_df = first_intime_per_subject.reset_index()

# Display the DataFrame with the first intime for each subject
print(first_intime_df)

      subject_id              intime
0       10002430 2129-06-13 00:43:08
1       10031757 2137-10-12 22:44:57
2       10032725 2143-03-22 06:42:00
3       10054464 2122-05-04 09:48:00
4       10063856 2174-03-08 01:04:16
...          ...                 ...
1269    19918048 2135-02-11 20:43:01
1270    19930554 2194-07-26 12:39:16
1271    19955909 2180-07-29 18:47:34
1272    19963140 2137-09-13 22:53:55
1273    19984860 2114-03-04 22:18:28

[1274 rows x 2 columns]


## Add survival days
Use addmittime, dischtime and deathtime.  
"In addition, we used data from patients within 24 h of admission to the ICU for the analysis. If the patient had multiple measurements within 24 h of admission to the ICU, the data from the first measurement were used."

In [15]:
#grouped_stays.head(20)
#grouped_stays[grouped_stays['subject_id'] ==  16602058]

In [16]:
# checking random
#survival_df[survival_df['subject_id'] ==  16602058]

In [17]:
df_sorted = filtered_stays_icd0.groupby('subject_id').apply(lambda x: x.sort_values(by='intime')).reset_index(drop=True)



# Group by subject_id and check the count of rows for each subject
subject_counts = df_sorted.groupby('subject_id').size()

# Function to calculate survival days based on the number of rows for each subject
def calculate_survival(row):
    if subject_counts[row['subject_id']] == 1:
        return (row['dischtime'] - row['admittime']).days
    else:
        if pd.notnull(row['deathtime']):
            return (row['deathtime'] - row['admittime']).days
        else:
            return (row['dischtime'] - row['admittime']).days

agg_df = df_sorted.groupby('subject_id').agg({
    'admittime': 'min',
    'dischtime': 'max',
    'deathtime': 'max'
}).reset_index()

# Apply the function to calculate survival days
agg_df ['survival_days'] = agg_df .apply(calculate_survival, axis=1)

# Drop duplicate rows for subjects with multiple records
agg_df.drop_duplicates(subset='subject_id', keep='first', inplace=True)

# Create a new DataFrame with subject_id and survival_days
survival_df = agg_df [['subject_id', 'survival_days']].copy()

print(survival_df)



      subject_id  survival_days
0       10002430             11
1       10031757             11
2       10032725              3
3       10054464             38
4       10063856              6
...          ...            ...
1269    19918048              9
1270    19930554              3
1271    19955909              3
1272    19963140              9
1273    19984860              8

[1274 rows x 2 columns]


In [18]:

# agg_df = df_sorted.groupby('subject_id').agg({
#     'admittime': 'min',
#     'dischtime': 'max',
#     'deathtime': 'max'
# }).reset_index()


# agg_df[agg_df['subject_id'] ==  16602058]

## Get the other secondary cancers information from icd data

Respiratory system https://www.icd10data.com/ICD10CM/Codes/C00-D49/C76-C80/C78-/C78.30  
Digestive system https://www.icd10data.com/ICD10CM/Codes/C00-D49/D00-D09/D01-/D01.9  
https://www.icd10data.com/ICD10CM/Codes/C00-D49/C7B-C7B/C7B-/C7B.02  
Reproductive organ of the male https://www.icd10data.com/ICD10CM/Codes/C00-D49/D00-D09/D07-/D07.60  
https://www.icd10data.com/ICD10CM/Codes/C00-D49/C76-C80/C79-/C79.82  
Urinary system https://www.icd10data.com/ICD10CM/Codes/C00-D49/C76-C80/C79-/C79.19  
Breast https://www.icd10data.com/ICD10CM/Codes/C00-D49/C76-C80/C79-/C79.81  
Lymphatic and hematopoietic systems https://www.icd10data.com/ICD10CM/Codes/C00-D49/D37-D48/D47-/D47.Z9  
Skin and soft tissue https://www.icd10data.com/ICD10CM/Codes/C00-D49/C45-C49/C49-/C49.9  
Reproductive organ of the female https://www.icd10data.com/ICD10CM/Codes/C00-D49/C51-C58/C57-/C57.9  
Ill-defined, unspecified sites https://www.icd10data.com/ICD10CM/Codes/C00-D49/C76-C80/C76-/C76.8  



In [19]:
# Dictionary to map ICD code ranges to cancer types
cancer_types = {
    'Respiratory system': ['180', '181', '182', 'C78.0','C781','C782','C783','D70'],
    'Digestive system': ['374', '375', '376', '435', '436', '437', 'D01','C784','C785','C786','C787','C788'],
    'Reproductive organ of the male': ['715', '716', '722', '723', '724', 'C7982'],
    'Urinary system': ['656', '657', '658', '686', '687', '688','C7911'],
    'Breast': ['582', '583', '597', '598', '599', 'C7981'],
    'Lymphatic and hematopoietic systems': ['820', '821', '822', '823', '824', '825', '840', '841', '842', 'C47'],
    'Skin and soft tissue': ['542', '543', '544', 'C792'],
    'Reproductive organ of the female': ['736', '737', '738', '738', '739', '740', '741', '754', '755', '756', 'C57','C7982'],
    'Ill-defined, unspecified sites': ['826', '827', '828', '829', '830', '843', '844', '845', 'C76','C80']
}


In [20]:
def categorize_cancer_type(icd_code):
    for cancer_type, code_range in cancer_types.items():
        if icd_code.startswith(tuple(code_range)):
            return cancer_type
    return 'Unknown'


diagnoses_icd['cancer_type'] = diagnoses_icd['icd_code'].apply(categorize_cancer_type)


print(diagnoses_icd[['subject_id', 'icd_code', 'cancer_type']])

         subject_id icd_code cancer_type
0          10000032     5723     Unknown
1          10000032    78959     Unknown
2          10000032     5715     Unknown
3          10000032    07070     Unknown
4          10000032      496     Unknown
...             ...      ...         ...
4756321    19999987    41401     Unknown
4756322    19999987    78039     Unknown
4756323    19999987     0413     Unknown
4756324    19999987    36846     Unknown
4756325    19999987     7810     Unknown

[4756326 rows x 3 columns]


In [21]:
cancer_type_list = [
    'Respiratory system', 'Digestive system', 'Reproductive organ of the male',
    'Urinary system', 'Breast', 'Lymphatic and hematopoietic systems',
    'Skin and soft tissue', 'Reproductive organ of the female', 'Ill-defined, unspecified sites'
]


cancer_type_pivot = pd.pivot_table(
    diagnoses_icd,
    index='subject_id',
    columns='cancer_type',
    values='icd_code',
    aggfunc=lambda x: 1 if x.notna().any() else 0
)


cancer_type_pivot = cancer_type_pivot[cancer_type_list]

cancer_type_pivot

cancer_type,Respiratory system,Digestive system,Reproductive organ of the male,Urinary system,Breast,Lymphatic and hematopoietic systems,Skin and soft tissue,Reproductive organ of the female,"Ill-defined, unspecified sites"
subject_id,,,,,,,,,
10000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
19999733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19999784,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19999828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:

df_merged0=cancer_type_pivot.merge(first_intime_df, on='subject_id', how='inner')
df_merged0

,subject_id,Respiratory system,Digestive system,Reproductive organ of the male,Urinary system,Breast,Lymphatic and hematopoietic systems,Skin and soft tissue,Reproductive organ of the female,"Ill-defined, unspecified sites",intime
0,10002430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2129-06-13 00:43:08
1,10031757,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2137-10-12 22:44:57
2,10032725,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2143-03-22 06:42:00
3,10054464,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2122-05-04 09:48:00
4,10063856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2174-03-08 01:04:16
...,...,...,...,...,...,...,...,...,...,...,...
1269,19918048,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2135-02-11 20:43:01
1270,19930554,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,2194-07-26 12:39:16
1271,19955909,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2180-07-29 18:47:34
1272,19963140,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2137-09-13 22:53:55


In [23]:

df_merged = df_merged0.merge(survival_df, on='subject_id', how='inner')
df_merged




,subject_id,Respiratory system,Digestive system,Reproductive organ of the male,Urinary system,Breast,Lymphatic and hematopoietic systems,Skin and soft tissue,Reproductive organ of the female,"Ill-defined, unspecified sites",intime,survival_days
0,10002430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2129-06-13 00:43:08,11
1,10031757,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2137-10-12 22:44:57,11
2,10032725,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2143-03-22 06:42:00,3
3,10054464,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2122-05-04 09:48:00,38
4,10063856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2174-03-08 01:04:16,6
...,...,...,...,...,...,...,...,...,...,...,...,...
1269,19918048,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2135-02-11 20:43:01,9
1270,19930554,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,2194-07-26 12:39:16,3
1271,19955909,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2180-07-29 18:47:34,3
1272,19963140,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2137-09-13 22:53:55,9


## BMI data

Get the BMI from omr data.  
PS: cannot open icu/chartevents.csv.gz file, since it is too large. I extracted BMI from omr data.

In [24]:
omr_filtered = omr[omr['subject_id'].isin(patients_with_bone_tumors)]


omr_filtered_sorted = omr_filtered.sort_values(by='chartdate')


first_bmi_values = omr_filtered_sorted[omr_filtered_sorted['result_name'] == 'BMI (kg/m2)'].groupby('subject_id')['result_value'].first()

first_bmi_values = first_bmi_values.rename('BMI')

first_bmi_values = first_bmi_values.reset_index()



print(first_bmi_values)

      subject_id   BMI
0       10002430  23.4
1       10013653  27.4
2       10021930  40.1
3       10026879  19.1
4       10034933  22.6
...          ...   ...
2446    19955909  19.7
2447    19963140  30.7
2448    19972371  24.6
2449    19982183  27.9
2450    19984860  25.8

[2451 rows x 2 columns]


In [25]:
omr_filtered = omr[omr['subject_id'].isin(patients_with_bone_tumors)]


omr_filtered_sorted = omr_filtered.sort_values(by='chartdate')


first_h_values = omr_filtered_sorted[omr_filtered_sorted['result_name'] == 'Height (Inches)'].groupby('subject_id')['result_value'].first()

first_h_values = first_h_values.rename('Height (Inches)')

first_h_values = first_h_values.reset_index()



print(first_h_values)

      subject_id Height (Inches)
0       10002430              66
1       10013653            70.5
2       10021930            64.5
3       10026879              67
4       10034933           70.25
...          ...             ...
2413    19955909              66
2414    19963140              72
2415    19972371              64
2416    19982183              66
2417    19984860              62

[2418 rows x 2 columns]


In [26]:
# pd.merge(df_merged, first_h_values, on='subject_id', how='left')['Height (Inches)'].isna().sum()

In [27]:
filtered_id0 = pd.merge(df_merged, first_bmi_values, on='subject_id', how='left')

filtered_id0

,subject_id,Respiratory system,Digestive system,Reproductive organ of the male,Urinary system,Breast,Lymphatic and hematopoietic systems,Skin and soft tissue,Reproductive organ of the female,"Ill-defined, unspecified sites",intime,survival_days,BMI
0,10002430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2129-06-13 00:43:08,11,23.4
1,10031757,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2137-10-12 22:44:57,11,NaN
2,10032725,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2143-03-22 06:42:00,3,NaN
3,10054464,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2122-05-04 09:48:00,38,28.8
4,10063856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2174-03-08 01:04:16,6,26.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269,19918048,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2135-02-11 20:43:01,9,22.5
1270,19930554,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,2194-07-26 12:39:16,3,23.2
1271,19955909,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2180-07-29 18:47:34,3,19.7
1272,19963140,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2137-09-13 22:53:55,9,30.7


## Redefine races

In [28]:
filtered_id = filtered_id0.merge(stays, on=['subject_id', 'intime'], how='inner')
filtered_id

,subject_id,Respiratory system,Digestive system,Reproductive organ of the male,Urinary system,Breast,Lymphatic and hematopoietic systems,Skin and soft tissue,Reproductive organ of the female,"Ill-defined, unspecified sites",...,race,gender,dod,anchor_age,anchor_year,yob,age,mortality_inunit,mortality,mortality_inhospital
0,10002430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,WHITE,M,2130-01-11,86,2125,2039,90,0,0,0
1,10031757,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,WHITE,F,2137-10-31,67,2137,2070,67,0,0,0
2,10032725,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,BLACK/AFRICAN AMERICAN,F,2143-03-30,38,2143,2105,38,0,0,0
3,10054464,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,WHITE,M,2122-06-11,70,2122,2052,70,0,0,0
4,10063856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,WHITE,F,2179-03-25,54,2172,2118,56,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269,19918048,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,WHITE,M,2136-10-04,61,2135,2074,61,0,0,0
1270,19930554,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,...,BLACK/AFRICAN AMERICAN,F,2198-08-22,35,2194,2159,35,0,0,0
1271,19955909,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,ASIAN - CHINESE,M,2181-05-16,71,2180,2109,71,0,0,0
1272,19963140,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,WHITE,M,NaT,68,2137,2069,68,0,0,0


In [29]:
filtered_id['race'].unique()

array(['WHITE', 'BLACK/AFRICAN AMERICAN', 'WHITE - EASTERN EUROPEAN',
       'PATIENT DECLINED TO ANSWER', 'ASIAN', 'WHITE - OTHER EUROPEAN',
       'WHITE - RUSSIAN', 'HISPANIC/LATINO - PUERTO RICAN',
       'ASIAN - SOUTH EAST ASIAN', 'ASIAN - CHINESE', 'OTHER',
       'PORTUGUESE', 'BLACK/CAPE VERDEAN', 'BLACK/CARIBBEAN ISLAND',
       'UNKNOWN', 'HISPANIC OR LATINO', 'HISPANIC/LATINO - COLUMBIAN',
       'WHITE - BRAZILIAN', 'HISPANIC/LATINO - GUATEMALAN',
       'HISPANIC/LATINO - DOMINICAN', 'UNABLE TO OBTAIN',
       'SOUTH AMERICAN', 'BLACK/AFRICAN',
       'HISPANIC/LATINO - CENTRAL AMERICAN', 'MULTIPLE RACE/ETHNICITY',
       'HISPANIC/LATINO - CUBAN', 'HISPANIC/LATINO - HONDURAN',
       'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER',
       'HISPANIC/LATINO - MEXICAN'], dtype=object)

In [30]:
# Unique values of race
race_values = filtered_id['race'].unique()

# Define a function to categorize ethnicity
def categorize_ethnicity(race):
    if 'ASIAN' in race.upper():
        return 'Asian'
    elif 'BLACK' in race.upper():
        return 'Black'
    elif 'WHITE' in race.upper():
        return 'White'
    else:
        return 'Other'

# Create a new variable 'Ethnicity_Group' based on 'race'
filtered_id['Ethnicity'] = filtered_id['race'].apply(categorize_ethnicity)

# Display the DataFrame with the new variable
print(filtered_id[['race', 'Ethnicity']])




                        race Ethnicity
0                      WHITE     White
1                      WHITE     White
2     BLACK/AFRICAN AMERICAN     Black
3                      WHITE     White
4                      WHITE     White
...                      ...       ...
1269                   WHITE     White
1270  BLACK/AFRICAN AMERICAN     Black
1271         ASIAN - CHINESE     Asian
1272                   WHITE     White
1273      HISPANIC OR LATINO     Other

[1274 rows x 2 columns]


## Redefine genders

In [31]:
# Replace 'M' with 'Male' and 'F' with 'Female' in the 'gender' column
filtered_id['gender'] = filtered_id['gender'].replace({'M': 'Male', 'F': 'Female'})

# Display the updated DataFrame
print(filtered_id['gender'])

0         Male
1       Female
2       Female
3         Male
4       Female
         ...  
1269      Male
1270    Female
1271      Male
1272      Male
1273    Female
Name: gender, Length: 1274, dtype: object


In [32]:
filtered_id

,subject_id,Respiratory system,Digestive system,Reproductive organ of the male,Urinary system,Breast,Lymphatic and hematopoietic systems,Skin and soft tissue,Reproductive organ of the female,"Ill-defined, unspecified sites",...,gender,dod,anchor_age,anchor_year,yob,age,mortality_inunit,mortality,mortality_inhospital,Ethnicity
0,10002430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Male,2130-01-11,86,2125,2039,90,0,0,0,White
1,10031757,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,Female,2137-10-31,67,2137,2070,67,0,0,0,White
2,10032725,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,Female,2143-03-30,38,2143,2105,38,0,0,0,Black
3,10054464,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,Male,2122-06-11,70,2122,2052,70,0,0,0,White
4,10063856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Female,2179-03-25,54,2172,2118,56,0,0,0,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269,19918048,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,Male,2136-10-04,61,2135,2074,61,0,0,0,White
1270,19930554,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,...,Female,2198-08-22,35,2194,2159,35,0,0,0,Black
1271,19955909,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Male,2181-05-16,71,2180,2109,71,0,0,0,Asian
1272,19963140,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Male,NaT,68,2137,2069,68,0,0,0,White


## Table.

Variable checking

In [33]:
filtered_id.nunique()

subject_id                             1274
Respiratory system                        1
Digestive system                          1
Reproductive organ of the male            1
Urinary system                            1
Breast                                    1
Lymphatic and hematopoietic systems       1
Skin and soft tissue                      1
Reproductive organ of the female          1
Ill-defined, unspecified sites            1
intime                                 1274
survival_days                           275
BMI                                     268
hadm_id                                1274
stay_id                                1274
first_careunit                            9
last_careunit                             9
outtime                                1274
los                                    1271
admittime                              1274
dischtime                              1274
deathtime                               276
race                            

In [34]:
filtered_id['survival_days'].describe()

count    1274.000000
mean      141.313972
std       431.092726
min        -1.000000
25%         5.000000
50%        10.000000
75%        28.000000
max      3939.000000
Name: survival_days, dtype: float64

In [35]:
filtered_id['survival_days'].isna().sum()

0

In [36]:

filtered_id[filtered_id['survival_days'] == -1.000000]


,subject_id,Respiratory system,Digestive system,Reproductive organ of the male,Urinary system,Breast,Lymphatic and hematopoietic systems,Skin and soft tissue,Reproductive organ of the female,"Ill-defined, unspecified sites",...,gender,dod,anchor_age,anchor_year,yob,age,mortality_inunit,mortality,mortality_inhospital,Ethnicity
131,11042406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Male,2131-01-31,58,2130,2072,59,0,0,0,White
943,17631949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Female,2181-03-26,62,2181,2119,62,0,0,0,White


In [37]:
print(filtered_stays_icd0[filtered_stays_icd0['subject_id'] == 11042406][['admittime','dischtime','deathtime']])
print(filtered_stays_icd0[filtered_stays_icd0['subject_id'] == 17631949][['admittime','dischtime','deathtime']])


              admittime           dischtime           deathtime
532 2131-01-31 15:08:00 2131-01-31 03:11:00 2131-01-31 03:11:00
               admittime           dischtime           deathtime
3397 2181-03-26 14:59:00 2181-03-26 00:05:00 2181-03-26 00:05:00
3398 2181-03-26 14:59:00 2181-03-26 00:05:00 2181-03-26 00:05:00


The data seems incorrect; drop this two records.

In [38]:
table_data1 = filtered_id.drop(filtered_id[(filtered_id['subject_id'] == 11042406) | (filtered_id['subject_id'] == 17631949)].index)
table_data1

,subject_id,Respiratory system,Digestive system,Reproductive organ of the male,Urinary system,Breast,Lymphatic and hematopoietic systems,Skin and soft tissue,Reproductive organ of the female,"Ill-defined, unspecified sites",...,gender,dod,anchor_age,anchor_year,yob,age,mortality_inunit,mortality,mortality_inhospital,Ethnicity
0,10002430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Male,2130-01-11,86,2125,2039,90,0,0,0,White
1,10031757,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,Female,2137-10-31,67,2137,2070,67,0,0,0,White
2,10032725,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,Female,2143-03-30,38,2143,2105,38,0,0,0,Black
3,10054464,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,Male,2122-06-11,70,2122,2052,70,0,0,0,White
4,10063856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Female,2179-03-25,54,2172,2118,56,0,0,0,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269,19918048,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,Male,2136-10-04,61,2135,2074,61,0,0,0,White
1270,19930554,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,...,Female,2198-08-22,35,2194,2159,35,0,0,0,Black
1271,19955909,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Male,2181-05-16,71,2180,2109,71,0,0,0,Asian
1272,19963140,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Male,NaT,68,2137,2069,68,0,0,0,White


In [39]:
table_data1['survival_days'].describe()

count    1272.000000
mean      141.537736
std       431.394774
min         0.000000
25%         5.000000
50%        10.000000
75%        28.000000
max      3939.000000
Name: survival_days, dtype: float64

In [40]:
table_data1['mortality'].isna().sum()

0

In [41]:
table_data1['gender'].isna().sum()

0

In [42]:
table_data1['BMI'].astype(float).describe()

count      1031.000000
mean        137.243259
std        3360.230555
min          11.600000
25%          23.500000
50%          27.300000
75%          31.550000
max      107840.200000
Name: BMI, dtype: float64

In [43]:
table_data1['BMI'].isna().sum()

241

## BMI processing

There are some 'bad BMI data' by extracting from omr, filter those unreal for human.
Also there are 18.9% missing data, so I choose to filter all those higher than 50 (Normally, BMI=30, are defined to be overweighted)

In [44]:
filtered_high_bmi = table_data1[table_data1['BMI'].astype(float) > 50]
corresponding_ids = filtered_high_bmi['subject_id']
corresponding_ids

284     12327925
527     14441563
772     16379320
797     16568313
847     16916629
855     16949818
868     17086592
1064    18461566
1073    18497108
1100    18669211
1122    18860797
1163    19106574
1212    19477649
Name: subject_id, dtype: int64

In [45]:
table_data2 =  table_data1[~table_data1['subject_id'].isin(corresponding_ids)]

table_data2

,subject_id,Respiratory system,Digestive system,Reproductive organ of the male,Urinary system,Breast,Lymphatic and hematopoietic systems,Skin and soft tissue,Reproductive organ of the female,"Ill-defined, unspecified sites",...,gender,dod,anchor_age,anchor_year,yob,age,mortality_inunit,mortality,mortality_inhospital,Ethnicity
0,10002430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Male,2130-01-11,86,2125,2039,90,0,0,0,White
1,10031757,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,Female,2137-10-31,67,2137,2070,67,0,0,0,White
2,10032725,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,Female,2143-03-30,38,2143,2105,38,0,0,0,Black
3,10054464,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,Male,2122-06-11,70,2122,2052,70,0,0,0,White
4,10063856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Female,2179-03-25,54,2172,2118,56,0,0,0,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269,19918048,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,Male,2136-10-04,61,2135,2074,61,0,0,0,White
1270,19930554,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,...,Female,2198-08-22,35,2194,2159,35,0,0,0,Black
1271,19955909,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Male,2181-05-16,71,2180,2109,71,0,0,0,Asian
1272,19963140,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Male,NaT,68,2137,2069,68,0,0,0,White


In [46]:
bmi_fill_value =  table_data2['BMI'].astype(float).median()

bmi_fill_value

table_data2 = table_data2.copy()
table_data2['BMI'].fillna(bmi_fill_value, inplace=True)

In [47]:
bmi_summary = table_data2['BMI'].astype(float).describe()
bmi_summary_format = f"{bmi_summary['50%']:.2f} [{bmi_summary['min']:.2f}, {bmi_summary['max']:.2f}]"
bmi_summary_format

'27.20 [11.60, 48.50]'

In [48]:
table_data2['BMI'].astype(float).describe()

count    1259.000000
mean       27.780620
std         5.463309
min        11.600000
25%        24.400000
50%        27.200000
75%        30.200000
max        48.500000
Name: BMI, dtype: float64

## Table values

In [49]:
survival_summary = table_data2['survival_days'].describe()
survival_time_format = f"{survival_summary['50%']:.2f} [{survival_summary['min']:.2f}, {survival_summary['max']:.2f}]"



mortality_percentage = (table_data2['mortality'].sum() / len(table_data2)) * 100
mortality_counts= table_data2['mortality'].sum()
mortality_percentage_format = f"{mortality_counts} ({mortality_percentage:.2f}%)"




gender_percentage = (table_data2['gender'] == 'Male').mean() * 100
gender_count = (table_data2['gender'] == 'Male').sum()
gender_format = f"{gender_count} ({gender_percentage:.2f}%)"



age_summary = table_data2['age'].describe()
age_time_format = f"{age_summary['50%']:.2f} [{age_summary['min']:.2f}, {age_summary['max']:.2f}]"



ethnicity_percentage = table_data2['Ethnicity'].value_counts(normalize=True) * 100
ethnicity_counts = table_data2['Ethnicity'].value_counts()
ethnicity_summary = pd.DataFrame({'Count': ethnicity_counts, 'Percentage': ethnicity_percentage})
ethnicity_summary['Formatted'] = [f"{count} ({percentage:.2f}%)" for count, percentage in zip(ethnicity_counts, ethnicity_percentage)]





bmi_summary = table_data2['BMI'].astype(float).describe()
bmi_summary_format = f"{bmi_summary['50%']:.2f} [{bmi_summary['min']:.2f}, {bmi_summary['max']:.2f}]"



Respiratory_system_p = table_data2['Respiratory system'].sum()/ len(table_data2) * 100
Respiratory_system_c = table_data2['Respiratory system'].sum()
Respiratory_system_format = f" {Respiratory_system_c} ({Respiratory_system_p:.2f}%)"



Digestive_system_p = table_data2['Digestive system'].sum() / len(table_data2) * 100
Digestive_system_c = table_data2['Digestive system'].sum()
Digestive_system_format = f"{Digestive_system_c} ({Digestive_system_p:.2f}%)"

Reproductive_organ_of_the_male_p = table_data2['Reproductive organ of the male'].sum() / len(table_data2) * 100
Reproductive_organ_of_the_male_c = table_data2['Reproductive organ of the male'].sum()
Reproductive_organ_of_the_male_format = f"{Reproductive_organ_of_the_male_c} ({Reproductive_organ_of_the_male_p:.2f}%)"

Urinary_system_p = table_data2['Urinary system'].sum() / len(table_data2) * 100
Urinary_system_c = table_data2['Urinary system'].sum()
Urinary_system_format = f"{Urinary_system_c} ({Urinary_system_p:.2f}%)"

Breast_p = table_data2['Breast'].sum() / len(table_data2) * 100
Breast_c = table_data2['Breast'].sum()
Breast_format = f"{Breast_c} ({Breast_p:.2f}%)"

Lymphatic_and_hematopoietic_systems_p = table_data2['Lymphatic and hematopoietic systems'].sum() / len(table_data2) * 100
Lymphatic_and_hematopoietic_systems_c = table_data2['Lymphatic and hematopoietic systems'].sum()
Lymphatic_and_hematopoietic_systems_format = f"{Lymphatic_and_hematopoietic_systems_c} ({Lymphatic_and_hematopoietic_systems_p:.2f}%)"

Skin_and_soft_tissue_p = table_data2['Skin and soft tissue'].sum() / len(table_data2) * 100
Skin_and_soft_tissue_c = table_data2['Skin and soft tissue'].sum()
Skin_and_soft_tissue_format = f"{Skin_and_soft_tissue_c} ({Skin_and_soft_tissue_p:.2f}%)"

Reproductive_organ_of_the_female_p = table_data2['Reproductive organ of the female'].sum() / len(table_data2) * 100
Reproductive_organ_of_the_female_c = table_data2['Reproductive organ of the female'].sum()
Reproductive_organ_of_the_female_format = f"{Reproductive_organ_of_the_female_c} ({Reproductive_organ_of_the_female_p:.2f}%)"

unspecified_sites_p = table_data2['Ill-defined, unspecified sites'].sum() / len(table_data2) * 100
unspecified_sites_c = table_data2['Ill-defined, unspecified sites'].sum()
unspecified_sites_format = f"{unspecified_sites_c} ({unspecified_sites_p:.2f}%)"

x=' '




In [50]:
summary_table1 = pd.DataFrame({

    'Survival time (Days)': [survival_time_format],
    'Status = Dead (%)': [mortality_percentage_format],
    'Gender(male)': [gender_format],
    'Age (Years)': [age_time_format] ,
    'BMI (kg/m2)': [bmi_summary_format]
    })

summary_table2 = pd.DataFrame({

    'Ethnicity': ethnicity_summary['Formatted']
    })

summary_table3 = pd.DataFrame({

    'Type of cancer (%)': [x],
    'Respiratory System': Respiratory_system_format,
    'Digestive System': Digestive_system_format,
    'Reproductive Organ of the Male': Reproductive_organ_of_the_male_format,
    'Urinary System': Urinary_system_format,
    'Breast': Breast_format,
    'Lymphatic and Hematopoietic Systems': Lymphatic_and_hematopoietic_systems_format,
    'Skin and Soft Tissue': Skin_and_soft_tissue_format,
    'Reproductive Organ of the Female': Reproductive_organ_of_the_female_format,
    'Unspecified Sites': unspecified_sites_format
    })



summary_table1_transposed = summary_table1.T
summary_table2_transposed = summary_table2.T
summary_table3_transposed = summary_table3.T


print(summary_table1_transposed)
print(summary_table2_transposed)
print(summary_table3_transposed)


                                          0
Survival time (Days)  10.00 [0.00, 3939.00]
Status = Dead (%)              276 (21.92%)
Gender(male)                   734 (58.30%)
Age (Years)           66.00 [22.00, 100.00]
BMI (kg/m2)            27.20 [11.60, 48.50]
Ethnicity         White         Black         Other       Asian
Ethnicity  912 (72.44%)  144 (11.44%)  140 (11.12%)  63 (5.00%)
                                                  0
Type of cancer (%)                                 
Respiratory System                    113.0 (8.98%)
Digestive System                     253.0 (20.10%)
Reproductive Organ of the Male       264.0 (20.97%)
Urinary System                         15.0 (1.19%)
Breast                               294.0 (23.35%)
Lymphatic and Hematopoietic Systems    21.0 (1.67%)
Skin and Soft Tissue                   14.0 (1.11%)
Reproductive Organ of the Female       44.0 (3.49%)
Unspecified Sites                      36.0 (2.86%)


## Q2.
Build the predictive models using the cases and controls you define base

In [51]:
### Quality check
stay_filtered [(stay_filtered ['mortality']==1) & (stay_filtered ['deathtime'].isna())] [['subject_id', 'hadm_id', 'stay_id',
       'intime', 'outtime', 'los', 'admittime', 'dischtime', 'deathtime',
       'race', 'gender', 'dod', 'anchor_age', 'anchor_year', 'yob', 'age',
       'mortality_inunit', 'mortality', 'mortality_inhospital']]

,subject_id,hadm_id,stay_id,intime,outtime,los,admittime,dischtime,deathtime,race,gender,dod,anchor_age,anchor_year,yob,age,mortality_inunit,mortality,mortality_inhospital
69,10288867,26870369,39812813,2196-09-09 02:47:00,2196-09-12 04:07:07,3.055637,2196-09-09 01:16:00,2196-09-13 20:03:00,NaT,HISPANIC/LATINO - PUERTO RICAN,M,2196-09-13,69,2193,2124,72,0,1,1
70,10288867,26870369,39812813,2196-09-09 02:47:00,2196-09-12 04:07:07,3.055637,2196-09-09 01:16:00,2196-09-13 20:03:00,NaT,HISPANIC/LATINO - PUERTO RICAN,M,2196-09-13,69,2193,2124,72,0,1,1
71,10288867,26870369,39812813,2196-09-09 02:47:00,2196-09-12 04:07:07,3.055637,2196-09-09 01:16:00,2196-09-13 20:03:00,NaT,HISPANIC/LATINO - PUERTO RICAN,M,2196-09-13,69,2193,2124,72,0,1,1
72,10288867,26870369,39812813,2196-09-09 02:47:00,2196-09-12 04:07:07,3.055637,2196-09-09 01:16:00,2196-09-13 20:03:00,NaT,HISPANIC/LATINO - PUERTO RICAN,M,2196-09-13,69,2193,2124,72,0,1,1
73,10288867,26870369,39812813,2196-09-09 02:47:00,2196-09-12 04:07:07,3.055637,2196-09-09 01:16:00,2196-09-13 20:03:00,NaT,HISPANIC/LATINO - PUERTO RICAN,M,2196-09-13,69,2193,2124,72,0,1,1
74,10288867,26870369,39812813,2196-09-09 02:47:00,2196-09-12 04:07:07,3.055637,2196-09-09 01:16:00,2196-09-13 20:03:00,NaT,HISPANIC/LATINO - PUERTO RICAN,M,2196-09-13,69,2193,2124,72,0,1,1
1545,13288288,23468411,31142097,2168-01-17 14:29:25,2168-01-21 02:15:04,3.490035,2168-01-17 14:28:00,2168-01-24 08:35:00,NaT,BLACK/AFRICAN AMERICAN,M,2168-01-24,54,2163,2109,59,0,1,1
1546,13288288,23468411,33208478,2168-01-22 03:25:32,2168-01-24 08:35:50,2.215486,2168-01-17 14:28:00,2168-01-24 08:35:00,NaT,BLACK/AFRICAN AMERICAN,M,2168-01-24,54,2163,2109,59,1,1,1
3382,17607166,26314500,35668995,2192-03-27 11:22:22,2192-03-31 04:01:36,3.693912,2192-03-26 00:34:00,2192-03-31 01:51:00,NaT,PORTUGUESE,M,2192-03-31,51,2187,2136,56,1,1,1
3383,17607166,26314500,35668995,2192-03-27 11:22:22,2192-03-31 04:01:36,3.693912,2192-03-26 00:34:00,2192-03-31 01:51:00,NaT,PORTUGUESE,M,2192-03-31,51,2187,2136,56,1,1,1


In [52]:
diagnosis = pd.read_csv(mimic4_path+'/hosp/diagnoses_icd.csv.gz', compression='gzip', header=0)
admis = pd.read_csv(mimic4_path+'/hosp/admissions.csv.gz', compression='gzip', header=0)
proc= pd.read_csv(mimic4_path+'/hosp/procedures_icd.csv.gz', compression='gzip', header=0)

In [53]:
diagnosis

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9
...,...,...,...,...,...
4756321,19999987,23865745,7,41401,9
4756322,19999987,23865745,8,78039,9
4756323,19999987,23865745,9,0413,9
4756324,19999987,23865745,10,36846,9


In [54]:
### Cases for patient level prediction using the last ICU stay
case_pts= stay_filtered[['subject_id','intime','dischtime','deathtime']][stay_filtered ['mortality']== 1]
case_pts['deathtime']=case_pts['deathtime'].fillna(case_pts['dischtime'])
case_pts_index= case_pts.groupby('subject_id')[['intime','deathtime']].max().reset_index()
case_pts_index['tte']=(pd.to_datetime(case_pts_index['deathtime'])-pd.to_datetime(case_pts_index['intime'])).dt.days
case_adm=admis[admis['subject_id'].isin(case_pts['subject_id'].drop_duplicates().tolist())]
case_adm1=pd.merge(case_pts_index,case_adm, right_on='subject_id', left_on='subject_id')
case_adm2=case_adm1[pd.to_datetime(case_adm1['admittime']) <= case_adm1['intime']] #1434 records

In [55]:
case_adm2

,subject_id,intime,deathtime_x,tte,hadm_id,admittime,dischtime,deathtime_y,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10054464,2122-06-11 15:06:00,2122-06-11 21:20:00,0,20069204,2122-05-04 09:01:00,2122-05-12 11:44:00,NaN,EW EMER.,P79SJ2,EMERGENCY ROOM,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,2122-05-04 06:15:00,2122-05-04 09:48:00,0
1,10054464,2122-06-11 15:06:00,2122-06-11 21:20:00,0,21293609,2122-06-04 20:20:00,2122-06-06 18:40:00,NaN,EW EMER.,P612YK,EMERGENCY ROOM,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,2122-06-04 16:43:00,2122-06-04 21:31:00,0
2,10054464,2122-06-11 15:06:00,2122-06-11 21:20:00,0,21915212,2122-04-10 02:21:00,2122-04-18 17:45:00,NaN,EW EMER.,P257ZL,EMERGENCY ROOM,HOME,Medicare,ENGLISH,MARRIED,WHITE,2122-04-09 20:35:00,2122-04-10 03:24:00,0
3,10054464,2122-06-11 15:06:00,2122-06-11 21:20:00,0,22824457,2122-04-23 08:55:00,2122-05-03 18:30:00,NaN,EW EMER.,P53RDR,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,WHITE,2122-04-23 04:56:00,2122-04-23 09:55:00,0
4,10054464,2122-06-11 15:06:00,2122-06-11 21:20:00,0,22926852,2122-06-11 13:04:00,2122-06-11 21:20:00,2122-06-11 21:20:00,EW EMER.,P57ON4,EMERGENCY ROOM,DIED,Other,ENGLISH,MARRIED,WHITE,2122-06-11 08:21:00,2122-06-11 15:06:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1430,19851462,2129-05-20 14:59:43,2129-05-22 18:05:00,2,23701382,2128-08-24 21:15:00,2128-09-11 19:25:00,NaN,URGENT,P91RC2,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,?,MARRIED,ASIAN,NaN,NaN,0
1431,19851462,2129-05-20 14:59:43,2129-05-22 18:05:00,2,24624491,2128-12-11 16:57:00,2128-12-14 16:05:00,NaN,EW EMER.,P818OH,EMERGENCY ROOM,HOME,Other,ENGLISH,MARRIED,ASIAN,2128-12-11 14:31:00,2128-12-11 19:09:00,0
1432,19851462,2129-05-20 14:59:43,2129-05-22 18:05:00,2,28852035,2129-05-14 18:48:00,2129-05-22 18:05:00,2129-05-22 18:05:00,EW EMER.,P818OH,EMERGENCY ROOM,DIED,Other,ENGLISH,MARRIED,ASIAN,2129-05-14 13:22:00,2129-05-14 19:50:00,1
1433,19851462,2129-05-20 14:59:43,2129-05-22 18:05:00,2,28920215,2128-09-22 18:36:00,2128-10-01 19:30:00,NaN,EW EMER.,P88KYP,EMERGENCY ROOM,HOME,Other,?,MARRIED,ASIAN,2128-09-22 15:58:00,2128-09-22 20:35:00,0


Quick Quality check

In [56]:
#case_adm2[case_adm2['admittime']>case_adm2['dischtime']] ### good
case_adm2[case_adm2['admittime']>case_adm2['deathtime_y']] ### good

,subject_id,intime,deathtime_x,tte,hadm_id,admittime,dischtime,deathtime_y,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag


In [57]:
### Extract diagnosis data for cases
case_diag=diagnosis[(diagnosis['subject_id'].isin(case_adm2['subject_id'].drop_duplicates().tolist()))&(diagnosis['hadm_id'].isin(case_adm2['hadm_id'].drop_duplicates().tolist()))]
case_diag=case_diag.merge(case_adm2[['subject_id','hadm_id','admittime','dischtime']].drop_duplicates(),how='left')
case_diag['dischtime']=pd.to_datetime(case_diag['dischtime']).dt.date
case_diag['event_code']='D_ICD'+case_diag['icd_version'].astype('str')+'_'+case_diag['icd_code']
case_diag=case_diag[['subject_id','event_code','dischtime']].drop_duplicates()

In [58]:
case_diag

,subject_id,event_code,dischtime
0,10054464,D_ICD9_41071,2122-05-12
1,10054464,D_ICD9_42822,2122-05-12
2,10054464,D_ICD9_1629,2122-05-12
3,10054464,D_ICD9_1987,2122-05-12
4,10054464,D_ICD9_1983,2122-05-12
...,...,...,...
23109,19860952,D_ICD9_3899,2163-07-08
23110,19860952,D_ICD9_28860,2163-07-08
23111,19860952,D_ICD9_78791,2163-07-08
23112,19860952,D_ICD9_V1271,2163-07-08


In [59]:
### Extract procedure data for cases
case_proc=proc[(proc['subject_id'].isin(case_adm2['subject_id'].drop_duplicates().tolist()))&(proc['hadm_id'].isin(case_adm2['hadm_id'].drop_duplicates().tolist()))]
case_proc=case_proc.merge(case_pts_index.drop_duplicates(),how='left')
case_proc=case_proc[pd.to_datetime(case_proc['chartdate']) <= case_proc['intime']] ### only to keep events occur before the index date
case_proc=case_proc.merge(case_adm2[['subject_id','hadm_id','admittime','dischtime']].drop_duplicates(),how='left')
case_proc['dischtime']=pd.to_datetime(case_proc['dischtime']).dt.date
case_proc['event_code']='P_ICD'+ case_proc['icd_version'].astype('str')+'_'+case_proc['icd_code']
case_proc=case_proc[['subject_id','event_code','dischtime']].drop_duplicates()

In [60]:
case_proc

,subject_id,event_code,dischtime
0,10054464,P_ICD9_8321,2122-04-18
1,10054464,P_ICD9_9605,2122-04-18
2,10054464,P_ICD9_3201,2122-04-18
3,10054464,P_ICD9_9229,2122-04-18
4,10054464,P_ICD9_9925,2122-04-18
...,...,...,...
2744,19851462,P_ICD9_9671,2129-05-22
2745,19851462,P_ICD9_9925,2128-10-01
2746,19860952,P_ICD9_0159,2163-07-08
2747,19860952,P_ICD9_4011,2163-07-08


In [61]:
### ctrls for patient level prediction using the last ICU stay

ctrl_pts= stay_filtered[['subject_id','intime','dischtime']][~(stay_filtered['subject_id'].isin(case_pts['subject_id'].drop_duplicates().tolist()))]
ctrl_pts.columns=['subject_id','intime','last_dischtime']
ctrl_pts_index= ctrl_pts.groupby('subject_id')[['intime','last_dischtime']].max().reset_index()
ctrl_pts_index['tte']=(pd.to_datetime(ctrl_pts_index['last_dischtime'])-pd.to_datetime(ctrl_pts_index['intime'])).dt.days
ctrl_adm=admis[admis['subject_id'].isin(ctrl_pts['subject_id'].drop_duplicates().tolist())]
ctrl_adm1=pd.merge(ctrl_pts_index,ctrl_adm, right_on='subject_id', left_on='subject_id')
ctrl_adm2=ctrl_adm1[pd.to_datetime(ctrl_adm1['admittime']) <= ctrl_adm1['intime']]

### Extract diagnosis data for ctrl

ctrl_diag=diagnosis[(diagnosis['subject_id'].isin(ctrl_adm2['subject_id'].drop_duplicates().tolist()))&(diagnosis['hadm_id'].isin(ctrl_adm2['hadm_id'].drop_duplicates().tolist()))]
ctrl_diag=ctrl_diag.merge(ctrl_adm2[['subject_id','hadm_id','admittime','dischtime']].drop_duplicates(),how='left')
ctrl_diag['dischtime']=pd.to_datetime(ctrl_diag['dischtime']).dt.date
ctrl_diag['event_code']='D_ICD'+ctrl_diag['icd_version'].astype('str')+'_'+ctrl_diag['icd_code']
ctrl_diag=ctrl_diag[['subject_id','event_code','dischtime']].drop_duplicates()

### Extract procedure data for ctrl
ctrl_proc=proc[(proc['subject_id'].isin(ctrl_adm2['subject_id'].drop_duplicates().tolist()))&(proc['hadm_id'].isin(ctrl_adm2['hadm_id'].drop_duplicates().tolist()))]
ctrl_proc=ctrl_proc.merge(ctrl_pts_index.drop_duplicates(),how='left')
ctrl_proc=ctrl_proc[pd.to_datetime(ctrl_proc['chartdate']) <= ctrl_proc['intime']] ### only to keep events occur before the index date
ctrl_proc=ctrl_proc.merge(ctrl_adm2[['subject_id','hadm_id','admittime','dischtime']].drop_duplicates(),how='left')
ctrl_proc['dischtime']=pd.to_datetime(ctrl_proc['dischtime']).dt.date
ctrl_proc['event_code']='P_ICD'+ ctrl_proc['icd_version'].astype('str')+'_'+ctrl_proc['icd_code']
ctrl_proc=ctrl_proc[['subject_id','event_code','dischtime']].drop_duplicates()


In [62]:
### dump extracted data
pd.concat([case_diag,
           case_proc,
           ctrl_diag,
           ctrl_proc]
          ).dropna().to_csv(output_path+'/Mimic_PT_mortality_data_dp.tsv', sep='\t',index=False)

### dump labels, mortality label and time to death for survival

ctrl_pts_index['mort']=0
case_pts_index['mort']=1

pd.concat([ctrl_pts_index[['subject_id','mort','tte']].drop_duplicates(),
           case_pts_index[['subject_id','mort','tte']].drop_duplicates()
           ]).dropna().to_csv(output_path+'/Mimic_PT_mortality_labels.tsv', sep='\t',index=False)

## Proprocess data into pickled list

In [63]:
sys.path.insert(0,"/content/drive/MyDrive/pytorch_ehr/Pytorch_EHR_Tutorial/Data_Prep/")
from preprocess_outcomes  import dump_split_process_data

In [64]:
output_proc='/content/drive/MyDrive/pytorch_ehr/Pytorch_EHR_Tutorial/Data_Prep/data/processed_data'
if not os.path.exists(output_proc):
        os.makedirs(output_proc)

dump_split_process_data(output_path+'/Mimic_PT_mortality_data_dp.tsv', output_path+'/Mimic_PT_mortality_labels.tsv' , 'NA' ,output_proc+'/Mimic_PT_mortality_dp_v1' , 'NA' )

loading data
loaded data for:  1274
loading labels
loaded labels for:  1274  after primary cleaning  1274
Mortality Case counts:  348
LOS>7 :  402
new types dictionary
processed 1000 pts
Reparsing
Splitting
random split


In [65]:
### Extract code descriptions as we will use later

diag_desc = pd.read_csv(mimic4_path+'/hosp/d_icd_diagnoses.csv.gz', compression='gzip', header=0)
diag_desc['event_code']='D_ICD'+diag_desc['icd_version'].astype('str')+'_'+diag_desc['icd_code']
proc_desc= pd.read_csv(mimic4_path+'/hosp/d_icd_procedures.csv.gz', compression='gzip', header=0)
proc_desc['event_code']='P_ICD'+proc_desc['icd_version'].astype('str')+'_'+proc_desc['icd_code']
diag_desc['cat']='Diag'
proc_desc['cat']='Proc'
code_desc=pd.concat([diag_desc[['event_code','cat','long_title']],proc_desc[['event_code','cat','long_title']]])
code_desc.to_csv(output_proc+'/code_desc.tsv',sep='\t',index=False)

## Data in OMOP format (MIMIC IV Demo Data)

> First we need to define our objective:
  This is a demo data of 100 patients, so we will use all patients till last admission to predict mortality (so note the difference from the previous cohort, is that we don't retrict to icu stays and it is not just in-hospital mortality , i.e we wase our conditions for the sake of the Demo)

Again we will prepare to predict both binary and survival

So we need a label file that include: 'subject_id' , 'mortality_indicator' , 'time from index date to death'

and then extract all patient diagnosis and procedures available till the index date and use the linked visit admission or discharge date to later define the sequence of visits

  So our steps:

  1. Identify the patients label (died or not) --> Death Table
  2. Find out the index date per patient (admission date for last encounter/visit) ----> visit_occurence
  3. Extract diagnosis and procedures, along with the linked visit admission or discharge date

In [66]:
mimic4_omop_pth='/content/drive/MyDrive/pytorch_ehr/Pytorch_EHR_Tutorial/Data_Prep/data/physionet.org/files/mimic-iv-demo-omop/0.9/1_omop_data_csv/'
mo_db_pth='/content/drive/MyDrive/pytorch_ehr/Pytorch_EHR_Tutorial/Data_Prep/data/physionet.org/files/'

Let's understand the data First

In [67]:
pt_died = pd.read_csv(mimic4_omop_pth+'death.csv', header=0)
pt_died =pt_died[['person_id','death_date']]
pt_died

,person_id,death_date
0,-2312013739856114142,2116-07-05
1,-7671795861352464589,2115-10-12
2,1194579079287927665,2177-03-29
3,579254014084392336,2117-03-24
4,-4353160957725823366,2146-07-12
5,4352191084057402257,2135-01-19
6,2213013192725646311,2186-11-17
7,6339505631013617478,2111-11-15
8,-3908355835367628651,2137-09-02
9,4783904755296699562,2116-03-12


In [68]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
pts = pd.read_csv(mimic4_omop_pth+'person.csv', header=0)
pts


,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,race_concept_id,ethnicity_concept_id,location_id,provider_id,care_site_id,person_source_value,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,ethnicity_source_value,ethnicity_source_concept_id
0,3589912774911670296,8507,2095,NaN,NaN,NaN,0,38003563,NaN,NaN,NaN,10009628,M,0,NaN,0,HISPANIC/LATINO,2000001408
1,-3210373572193940939,8507,2079,NaN,NaN,NaN,0,38003563,NaN,NaN,NaN,10011398,M,0,NaN,0,HISPANIC/LATINO,2000001408
2,-775517641933593374,8507,2149,NaN,NaN,NaN,8516,0,NaN,NaN,NaN,10004235,M,0,BLACK/AFRICAN AMERICAN,2000001406,NaN,0
3,-2575767131279873665,8507,2050,NaN,NaN,NaN,8516,0,NaN,NaN,NaN,10024043,M,0,BLACK/AFRICAN AMERICAN,2000001406,NaN,0
4,-8970844422700220177,8507,2114,NaN,NaN,NaN,8527,0,NaN,NaN,NaN,10038933,M,0,WHITE,2000001404,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-7671795861352464589,8532,2052,NaN,NaN,NaN,2000001401,0,NaN,NaN,NaN,10038081,F,0,UNKNOWN,2000001401,NaN,0
96,5734523979606454056,8532,2069,NaN,NaN,NaN,2000001401,0,NaN,NaN,NaN,10036156,F,0,UNKNOWN,2000001401,NaN,0
97,1532249960797525190,8532,2106,NaN,NaN,NaN,2000001405,0,NaN,NaN,NaN,10014078,F,0,UNABLE TO OBTAIN,2000001405,NaN,0
98,5894416985828315484,8532,2055,NaN,NaN,NaN,2000001405,0,NaN,NaN,NaN,10019172,F,0,UNABLE TO OBTAIN,2000001405,NaN,0


In [70]:
vts = pd.read_csv(mimic4_omop_pth+'visit_occurrence.csv', header=0)
vts[['person_id','visit_occurrence_id','visit_start_datetime','visit_end_datetime','admitting_source_value','discharge_to_source_value','preceding_visit_occurrence_id']]

,person_id,visit_occurrence_id,visit_start_datetime,visit_end_datetime,admitting_source_value,discharge_to_source_value,preceding_visit_occurrence_id
0,4783904755296699562,-4406053801395356975,2112-11-06 11:05:00,2112-11-06 11:05:00,NaN,NaN,-3.100296e+18
1,-6225647829918357531,2636026522589494723,2153-10-17 14:23:00,2153-10-17 14:23:00,NaN,NaN,-2.238366e+18
2,7918537411740862407,-8900473297538187901,2130-10-10 16:20:00,2130-10-10 16:20:00,NaN,NaN,-8.600570e+18
3,-6225647829918357531,-8091470952174089624,2155-04-20 09:00:00,2155-04-20 09:00:00,NaN,NaN,4.782862e+18
4,7155255168997124770,8144493519470334167,2147-07-08 08:20:00,2147-07-08 08:20:00,NaN,NaN,-3.275690e+18
...,...,...,...,...,...,...,...
847,4498126063475867818,6557338641781338493,2156-05-11 11:29:00,2156-05-22 14:16:00,EMERGENCY ROOM,CHRONIC/LONG TERM ACUTE CARE,NaN
848,4498126063475867818,-9026896896997717305,2156-04-30 18:30:00,2156-05-03 16:36:00,EMERGENCY ROOM,CHRONIC/LONG TERM ACUTE CARE,NaN
849,6128703162302148003,5538617222786573932,2178-07-22 05:41:00,2178-07-25 16:30:00,EMERGENCY ROOM,CHRONIC/LONG TERM ACUTE CARE,NaN
850,-3908355835367628651,-4177473998334219433,2137-02-24 10:00:00,2137-03-19 15:45:00,TRANSFER FROM SKILLED NURSING FACILITY,CHRONIC/LONG TERM ACUTE CARE,NaN


In [71]:
diag = pd.read_csv(mimic4_omop_pth+'condition_occurrence.csv', header=0)
diag

,condition_occurrence_id,person_id,condition_concept_id,condition_start_date,condition_start_datetime,condition_end_date,condition_end_datetime,condition_type_concept_id,stop_reason,provider_id,visit_occurrence_id,visit_detail_id,condition_source_value,condition_source_concept_id,condition_status_source_value,condition_status_concept_id
0,7000818053728441484,1741351032930224901,196523,2179-07-24,2179-07-24 18:21:00,2179-07-28,2179-07-28 15:54:00,32821,NaN,NaN,-5779522865065417426,NaN,78791,44824628,NaN,NaN
1,-3514320024333679102,1741351032930224901,436659,2179-07-24,2179-07-24 18:21:00,2179-07-28,2179-07-28 15:54:00,32821,NaN,NaN,-5779522865065417426,NaN,2809,44828816,NaN,NaN
2,-6297447354934110463,1741351032930224901,435515,2179-07-24,2179-07-24 18:21:00,2179-07-28,2179-07-28 15:54:00,32821,NaN,NaN,-5779522865065417426,NaN,2761,44822952,NaN,NaN
3,3131132784874295309,1741351032930224901,436096,2179-07-24,2179-07-24 18:21:00,2179-07-28,2179-07-28 15:54:00,32821,NaN,NaN,-5779522865065417426,NaN,33829,44834629,NaN,NaN
4,2246709778332812674,1741351032930224901,440383,2179-07-24,2179-07-24 18:21:00,2179-07-28,2179-07-28 15:54:00,32821,NaN,NaN,-5779522865065417426,NaN,311,44833421,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16436,5160751222712405682,-1210657672714831684,4064452,2134-01-01,2134-01-01 08:00:00,2134-01-01,2134-01-01 08:00:00,32817,NaN,NaN,-3312868479314057914,NaN,AF (Atrial Fibrillation),2000031019,NaN,NaN
16437,7817722518091634880,-1210657672714831684,4064452,2134-01-01,2134-01-01 12:00:00,2134-01-01,2134-01-01 12:00:00,32817,NaN,NaN,-3312868479314057914,NaN,AF (Atrial Fibrillation),2000031019,NaN,NaN
16438,5631597446422224513,-1210657672714831684,4064452,2134-01-01,2134-01-01 14:00:00,2134-01-01,2134-01-01 14:00:00,32817,NaN,NaN,-3312868479314057914,NaN,AF (Atrial Fibrillation),2000031019,NaN,NaN
16439,8817407860694965134,-1210657672714831684,4064452,2134-01-01,2134-01-01 13:00:00,2134-01-01,2134-01-01 13:00:00,32817,NaN,NaN,-3312868479314057914,NaN,AF (Atrial Fibrillation),2000031019,NaN,NaN


#### Some data quality check

In [72]:
diag['condition_source_value']=diag['condition_source_value'].str.strip()
v_diag=diag[diag['condition_source_value'].str.strip().str.lower().str.startswith('v')][['condition_source_value','condition_type_concept_id']].drop_duplicates() #32821 are normal diagnosis, 32817 are those derived from ECG/notes
e_diag=diag[diag['condition_source_value'].str.strip().str.lower().str.startswith(('e0','e8','e9'))][['condition_source_value','condition_type_concept_id']].drop_duplicates() #32821 are normal diagnosis, 32817 are those derived from ECG/notes


In [73]:
diag_desc = pd.read_csv(mimic4_path+'/hosp/d_icd_diagnoses.csv.gz', compression='gzip', header=0)
v_diag=v_diag.merge(diag_desc, how='left' ,left_on='condition_source_value',right_on='icd_code')
v_diag[v_diag['icd_code'].duplicated(keep=False)]

,condition_source_value,condition_type_concept_id,icd_code,icd_version,long_title
3,V462,32821,V462,9.0,"Other dependence on machines, supplemental oxygen"
4,V462,32821,V462,10.0,Person on outside of car injured in collision ...
7,V641,32821,V641,9.0,Surgical or other procedure not carried out be...
8,V641,32821,V641,10.0,Passenger in heavy transport vehicle injured i...
9,V643,32821,V643,9.0,Procedure not carried out for other reasons
10,V643,32821,V643,10.0,Unspecified occupant of heavy transport vehicl...
14,V860,32821,V860,9.0,Estrogen receptor positive status [ER+]
15,V860,32821,V860,10.0,Driver of special all-terrain or other off-roa...
17,V090,32821,V090,9.0,Infection with microorganisms resistant to pen...
18,V090,32821,V090,10.0,Pedestrian injured in nontraffic accident invo...


In [74]:
e_diag=e_diag.merge(diag_desc, how='left' ,left_on='condition_source_value',right_on='icd_code')
e_diag[e_diag['icd_code'].duplicated(keep=False)]

,condition_source_value,condition_type_concept_id,icd_code,icd_version,long_title
12,E8809,32821,E8809,9,Accidental fall on or from other stairs or steps
13,E8809,32821,E8809,10,"Other disorders of plasma-protein metabolism, ..."
23,E8339,32821,E8339,9,Fall on stairs or ladders in water transport i...
24,E8339,32821,E8339,10,Other disorders of phosphorus metabolism
29,E8881,32821,E8881,9,Fall resulting in striking against other object
30,E8881,32821,E8881,10,Metabolic syndrome


In [75]:
pts_toexclude=diag[diag['condition_source_value'].isin(['E8809','E8339','E8881','V090','V860','V462','V641','V643'])]['person_id'].unique().tolist()
len(pts_toexclude)

11

In [76]:
pt_died[pt_died['person_id'].isin(pts_toexclude)] ### I will exclude 4 out of originally 14 cases :), It is a decision point, exclude the whole pt or just those diag records, I prefer to exclude the full patient if it is a small % but for the sake of the demo, I'll only drop those diag and similarly procedures

,person_id,death_date
2,1194579079287927665,2177-03-29
8,-3908355835367628651,2137-09-02
9,4783904755296699562,2116-03-12
13,-9066461348710750663,2185-01-22


In [77]:
proc_o = pd.read_csv(mimic4_omop_pth+'procedure_occurrence.csv', header=0)
proc_o[['person_id','procedure_source_value','procedure_datetime','visit_occurrence_id']]

,person_id,procedure_source_value,procedure_datetime,visit_occurrence_id
0,4783904755296699562,19301,2113-07-18 14:55:00,-433474223361412760
1,7918537411740862407,19303,2129-10-30 13:20:00,7730200099818586525
2,4498126063475867818,27235,2157-07-18 16:49:00,2514722389640694708
3,8090044958540695372,36252,2143-03-19 12:00:00,6734319289435533605
4,5863607150722936210,36830,2142-07-07 16:40:00,-5935156580867756808
...,...,...,...,...
18442,-3908355835367628651,BT1DYZZ,2137-09-02 17:05:00,-3115724380369486208
18443,-3908355835367628651,BT1FYZZ,2137-09-02 17:05:00,-3115724380369486208
18444,-6225647829918357531,D7021ZZ,2155-05-19 18:27:00,-6661057387050813606
18445,-6225647829918357531,D7021ZZ,2155-06-15 16:30:00,-8296465610050985060


In [78]:
proc_desc = pd.read_csv(mimic4_path+'/hosp/d_icd_procedures.csv.gz', compression='gzip', header=0)
#proc_desc['icd_code']=proc_desc['icd_code'].str.strip()
proc_codes=proc_o[['procedure_source_value','procedure_type_concept_id']].drop_duplicates() #32821 are normal diagnosis, 32817 are those derived from ECG/notes
proc_codes['procedure_source_value']=proc_codes['procedure_source_value'].str.strip()
proc_codes=proc_codes.merge(proc_desc, how='left' ,left_on='procedure_source_value',right_on='icd_code')
proc_codes

,procedure_source_value,procedure_type_concept_id,icd_code,icd_version,long_title
0,19301,32821,NaN,NaN,NaN
1,19303,32821,NaN,NaN,NaN
2,27235,32821,NaN,NaN,NaN
3,36252,32821,NaN,NaN,NaN
4,36830,32821,NaN,NaN,NaN
...,...,...,...,...,...
306,B54BZZA,32821,B54BZZA,10.0,Ultrasonography of Right Lower Extremity Veins...
307,BT1DYZZ,32821,BT1DYZZ,10.0,"Fluoroscopy of Right Kidney, Ureter and Bladde..."
308,BT1FYZZ,32821,BT1FYZZ,10.0,"Fluoroscopy of Left Kidney, Ureter and Bladder..."
309,D7021ZZ,32821,D7021ZZ,10.0,Beam Radiation of Spleen using Photons 1 - 10 MeV


In [79]:
proc_codes.describe(include='all')

,procedure_source_value,procedure_type_concept_id,icd_code,icd_version,long_title
count,311,311.000000,159,159.0,159
unique,311,NaN,159,NaN,159
top,19301,NaN,009600Z,NaN,Drainage of Cerebral Ventricle with Drainage D...
freq,1,NaN,1,NaN,1
mean,NaN,32826.141479,NaN,10.0,NaN
std,NaN,6.195674,NaN,0.0,NaN
min,NaN,32817.000000,NaN,10.0,NaN
25%,NaN,32821.000000,NaN,10.0,NaN
50%,NaN,32821.000000,NaN,10.0,NaN
75%,NaN,32833.000000,NaN,10.0,NaN


In [80]:
proc_desc[proc_desc['icd_code'].str.contains('36830')]

,icd_code,icd_version,long_title


In [81]:
proc[proc['icd_code'].str.contains('683')]['icd_code'].drop_duplicates()

3157      6831
4739      6839
7110      8683
666845    5683
Name: icd_code, dtype: object

**something is wrong mainly because those code are CPT not ICD procedures** :)

you can just google "procedure icd 36830" ---> again this is a decision point and for the sake of the demo, I will just ignore those, in real practice, I make sure I include them


And here, where terminology normalization will be helpful ---- https://github.com/ZhiGroup/terminology_representation

In [82]:
proc_codes=proc_codes.dropna()
proc_codes[proc_codes['icd_code'].duplicated(keep=False)] #### so no duplicates issue

,procedure_source_value,procedure_type_concept_id,icd_code,icd_version,long_title


##### Alternate way to prepocess the data using sql for example you can use the below code to create a sqlite database, and you can move to sql from there

In [83]:
import sqlite3
conn = sqlite3.connect(mo_db_pth+'mimic_omop.db')
pts[['person_id','year_of_birth','gender_source_value','race_source_value','ethnicity_source_value']].to_sql('pts_omop', conn , if_exists='replace', index=False)
vts[['person_id','visit_occurrence_id','visit_start_datetime','visit_end_datetime','admitting_source_value','discharge_to_source_value','preceding_visit_occurrence_id']].to_sql('visits_omop', conn , if_exists='replace', index=False)
pd.read_csv(mimic4_omop_pth+'death.csv', header=0).to_sql('death_omop', conn , if_exists='replace', index=False)
diag[['person_id','visit_occurrence_id','condition_source_value','condition_start_datetime','condition_end_datetime']].to_sql('diag_omop', conn , if_exists='replace', index=False)
proc_o[['person_id','visit_occurrence_id','procedure_source_value','procedure_datetime']].to_sql('proc_omop', conn , if_exists='replace', index=False)
diag_desc.to_sql('diag_desc', conn, if_exists='replace', index=False)
proc_desc.to_sql('diag_desc', conn, if_exists='replace', index=False)
conn.close()